# 지난 시간 복습

1. Backward 연산을 재귀 => 반복
2. Function subclass의 사용을 편리하게
3. Code 작동 Test 기법: UnitTest module
4. 입력 크기 확장  
  
2,3,4는 거의 정리할 내용이 없음.  
1은 CS 지식 정리할 겸, 조금 자세히 준비해서 PPT 만들었음.  
자세한 내용과 꼬리 재귀에 관한 내용은 notion에 정리.  

In [25]:
import numpy as np

def as_array(x):
    return np.array(x)

class Variable:
    def __init__(self, data):
        self.x = data
        self.creator = None
        self.dx = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.dx is None:
            self.dx = np.ones_like(self.x)
        funcs = [self.creator]
        while funcs:
            func = funcs.pop()
            cur_dx = func.backward()
            prev_dx = func.output.dx
            func.input.dx = cur_dx * prev_dx
            print(func.input.dx)
            if func.input.creator is not None:
                funcs.append(func.input.creator)
        return
    
class Function:
    def __call__(self, vars):
        if type(vars) == list:
            xs = [var.x for var in vars]
            ys = list(map(Variable, map(as_array, self.forward(xs))))
            for y in ys:
                y.set_creator(self)
        else:
            xs = vars.x
            ys = as_array(self.forward(xs))
            ys = Variable(ys)
            ys.set_creator(self)
        self.input = vars
        self.output = ys
        return ys
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self):
        return np.exp(self.input.x)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self):
        return 2 * self.input.x
    
def exp(x=None):
    return Exp()(x) if x is not None else Exp()

def square(x=None):
    return Square()(x) if x is not None else Square()    

class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return y,

In [26]:
x = Variable(np.array(0.5))

y = square(exp(square(x)))
y.backward()
x.dx

2.568050833375483
3.297442541400256
3.297442541400256


3.297442541400256

In [27]:
arr = np.array([0.1,0.2])
len(arr)

2

In [29]:
xs = [Variable(np.array(2.0)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.x)

5.0


# 이번주에는 총 3가지를 배운다
1. 가변 길이 인수(step11의 개선 편)
2. 가변 길이 인수(역전파)
3. 같은 변수 반복 사용

# Not Clone coding

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.creator = None
        self.grad = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            func = funcs.pop()
            x, y = func.input, func.output
            x.grad = func.backward(y.grad)
            if x.creator is not None:
                funcs.append(x.creator)

    def cleangrad(self):
        self.grad = None

class Function:
    def __call__(self, *inputs):
        ys = self.forward(*inputs)
        for y in ys:
            y = as_array(y)
            y = Variable(y)
            y.set_creator(self)
        self.inputs = inputs
        self.outputs = ys

    def forward(self):
        return NotImplementedError()

    def backward(self):
        return NotImplementedError()
    
class Add(Function):
    def forward(self, x0, x1):
        return x0 + x1,

    def backward(self, gy):
        return gy + gy

# 1. 가변 길이 인수(step11의 개선 편)

In [34]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable(object):
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function(object):
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, )

In [35]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5
